In [11]:
# This notebook takes original geo and removes meshids where concept 
# also falls into other category (e.g. animal models of disease, disease progression)

# remove entries with tree number starting with C23 as this category is methodology related

import pandas as pd
import numpy as np
import os

path = '../../data/final'
tree_file = 'mesh.pkl'
geo_file = 'geo.pkl'

save_to = '../../data/final/geo_filtered.pkl'

mesh_df = pd.read_pickle(os.path.join(path, tree_file))
mesh_df['aux'] = 1
tags_df = pd.read_pickle(os.path.join(path, geo_file))

mask_only_in = mesh_df.groupby(['mesh_heading', 'category']).aux.mean().reset_index()
mask_only_in_1 = (mask_only_in.groupby('mesh_heading').sum()==1).reset_index()
mask_only_in_1 = mask_only_in_1.loc[mask_only_in_1.aux==True,:]
mesh_headings_only_in_1 = mask_only_in_1.mesh_heading.to_list()
tags_df = tags_df[np.isin(tags_df.mesh_heading, mesh_headings_only_in_1)]

# remove C23
tags_df = pd.merge(tags_df, mesh_df[['mesh_id','mesh_treenumbers']], on='mesh_id', how='left')
tags_df = tags_df.loc[tags_df.mesh_treenumbers.str[:3]!='C23',:]
tags_df = tags_df.drop('mesh_treenumbers',axis=1)

tags_df.to_pickle(save_to)